In [17]:
import pandas as pd
import numpy as np
import glob
import os
from tqdm import tqdm

In [21]:
BASE_PATH = r"/home/poovarasan/Documents/Msc AI projects/AI in sustainability/ai_sustainbility_project/data"
HALF_HOURLY_PATH = BASE_PATH + "/halfhourly_dataset"
DAILY_PATH = BASE_PATH + "/daily_dataset"
HHBLOCK_PATH = BASE_PATH + "/hhblock_dataset"

In [22]:
print("Loading weather data...")

weather_hourly = pd.read_csv(BASE_PATH + "/weather_hourly_darksky.csv")
weather_daily = pd.read_csv(BASE_PATH + "/weather_daily_darksky.csv")

weather_hourly['time'] = pd.to_datetime(weather_hourly['time'])
weather_daily['time'] = pd.to_datetime(weather_daily['time'])

print("Weather loaded")
print(weather_hourly.shape)


Loading weather data...
Weather loaded
(21165, 12)


In [28]:
print("Loading household & ACORN data...")

acorn = pd.read_csv(BASE_PATH + "/acorn_details.csv", encoding='ISO-8859-1')
household_info = pd.read_csv(BASE_PATH + "/informations_households.csv")

print(acorn.head())
print(household_info.head())


Loading household & ACORN data...
  MAIN CATEGORIES CATEGORIES  REFERENCE  ACORN-A  ACORN-B  ACORN-C  ACORN-D  \
0      POPULATION        Age    Age 0-4     77.0     83.0     72.0    100.0   
1      POPULATION        Age   Age 5-17    117.0    109.0     87.0     69.0   
2      POPULATION        Age  Age 18-24     64.0     73.0     67.0    107.0   
3      POPULATION        Age  Age 25-34     52.0     63.0     62.0    197.0   
4      POPULATION        Age  Age 35-49    102.0    105.0     91.0    124.0   

   ACORN-E  ACORN-F  ACORN-G  ACORN-H  ACORN-I  ACORN-J  ACORN-K  ACORN-L  \
0    120.0     77.0     97.0     97.0     63.0    119.0     67.0    114.0   
1     94.0     95.0    102.0    106.0     67.0     95.0     64.0    108.0   
2    100.0     71.0     83.0     89.0     62.0    104.0    459.0     97.0   
3    151.0     66.0     90.0     88.0     63.0    132.0    145.0    109.0   
4    118.0     93.0    102.0    103.0     76.0    111.0     67.0     99.0   

   ACORN-M  ACORN-N  ACORN-O

In [29]:
print("Loading holiday data...")

holidays = pd.read_csv(BASE_PATH + "/uk_bank_holidays.csv")
holidays['Bank holidays'] = pd.to_datetime(holidays['Bank holidays'])

print(holidays.head())


Loading holiday data...
  Bank holidays                                          Type
0    2012-12-26                                    Boxing Day
1    2012-12-25                                 Christmas Day
2    2012-08-27                           Summer bank holiday
3    2012-05-06  Queen?s Diamond Jubilee (extra bank holiday)
4    2012-04-06          Spring bank holiday (substitute day)


In [30]:
print("Loading smart meter data (this will take time)...")

all_files = glob.glob(HALF_HOURLY_PATH + "/**/*.csv", recursive=True)

df_list = []

for file in tqdm(all_files):
    try:
        temp = pd.read_csv(file)
        
        # convert time
        temp['tstp'] = pd.to_datetime(temp['tstp'])
        
        # sample to reduce size (IMPORTANT)
        temp = temp.sample(frac=0.05, random_state=42)   # 5% sample
        
        df_list.append(temp)
        
    except Exception as e:
        print("Error reading:", file)

energy_df = pd.concat(df_list, ignore_index=True)

print("Smart meter data shape:", energy_df.shape)
energy_df.head()


Loading smart meter data (this will take time)...


  3%|▎         | 3/112 [00:02<01:19,  1.37it/s]/tmp/ipykernel_38001/804780727.py:9: DtypeWarning: Columns (0: energy(kWh/hh)) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(file)
  7%|▋         | 8/112 [00:06<01:24,  1.23it/s]/tmp/ipykernel_38001/804780727.py:9: DtypeWarning: Columns (0: energy(kWh/hh)) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(file)
 29%|██▊       | 32/112 [00:24<01:05,  1.22it/s]/tmp/ipykernel_38001/804780727.py:9: DtypeWarning: Columns (0: energy(kWh/hh)) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(file)
 37%|███▋      | 41/112 [00:32<00:58,  1.22it/s]/tmp/ipykernel_38001/804780727.py:9: DtypeWarning: Columns (0: energy(kWh/hh)) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(file)
 60%|█████▉    | 67/112 [00:53<00:34,  1.30it/s]/tmp/ipykernel_38001/804780727.py:9: Dtype

Smart meter data shape: (8390856, 3)


,LCLid,tstp,energy(kWh/hh)
0,MAC000640,2012-12-13 08:00:00,0.065
1,MAC000320,2012-10-19 10:00:00,0.133
2,MAC000525,2012-12-08 23:00:00,0.174
3,MAC000544,2013-07-17 21:00:00,0.177
4,MAC000673,2014-02-10 08:30:00,0.685


In [34]:
print("Cleaning energy data...")

# convert energy column to numeric safely
energy_df['energy(kWh/hh)'] = pd.to_numeric(
    energy_df['energy(kWh/hh)'], 
    errors='coerce'
)

# remove NaN after conversion
energy_df = energy_df.dropna(subset=['energy(kWh/hh)'])

# remove negative or zero values
energy_df = energy_df[energy_df['energy(kWh/hh)'] > 0]

# rename column
energy_df.rename(columns={'energy(kWh/hh)': 'energy'}, inplace=True)

print("Energy cleaned")
print(energy_df.head())


Cleaning energy data...
Energy cleaned
       LCLid                tstp  energy
0  MAC000640 2012-12-13 08:00:00   0.065
1  MAC000320 2012-10-19 10:00:00   0.133
2  MAC000525 2012-12-08 23:00:00   0.174
3  MAC000544 2013-07-17 21:00:00   0.177
4  MAC000673 2014-02-10 08:30:00   0.685


In [35]:
print("Creating time features...")

energy_df['hour'] = energy_df['tstp'].dt.hour
energy_df['day'] = energy_df['tstp'].dt.day
energy_df['month'] = energy_df['tstp'].dt.month
energy_df['weekday'] = energy_df['tstp'].dt.weekday
energy_df['date'] = energy_df['tstp'].dt.date

# weekend flag
energy_df['is_weekend'] = energy_df['weekday'].isin([5,6]).astype(int)


Creating time features...


In [36]:
print("Merging weather data...")

weather_hourly.rename(columns={'time':'tstp'}, inplace=True)

energy_df = pd.merge(
    energy_df,
    weather_hourly,
    on="tstp",
    how="left"
)

print("After weather merge:", energy_df.shape)


Merging weather data...
After weather merge: (8290709, 20)


In [37]:
print("Adding holiday feature...")

energy_df['date'] = pd.to_datetime(energy_df['date'])
holidays['date'] = holidays['Bank holidays']

energy_df['is_holiday'] = energy_df['date'].isin(holidays['date']).astype(int)


Adding holiday feature...


In [43]:
print("Cleaning duplicate columns...")

cols_to_drop = [
    'stdortou_x','acorn_x','acorn_grouped_x','file_x',
    'stdortou_y','acorn_y','acorn_grouped_y','file_y'
]

for c in cols_to_drop:
    if c in energy_df.columns:
        energy_df.drop(columns=c, inplace=True)

print("Remaining columns:")
print(energy_df.columns)


Cleaning duplicate columns...
Remaining columns:
Index(['lclid', 'tstp', 'energy', 'hour', 'day', 'month', 'weekday', 'date',
       'is_weekend', 'visibility', 'windbearing', 'temperature', 'dewpoint',
       'pressure', 'apparenttemperature', 'windspeed', 'preciptype', 'icon',
       'humidity', 'summary', 'is_holiday'],
      dtype='str')


In [44]:
print("Merging household info correctly...")

energy_df = pd.merge(
    energy_df,
    household_info,
    on="lclid",
    how="left"
)

print("After merge:", energy_df.shape)
energy_df.head()


Merging household info correctly...
After merge: (8290709, 25)


,lclid,tstp,energy,hour,day,month,weekday,date,is_weekend,visibility,...,windspeed,preciptype,icon,humidity,summary,is_holiday,stdortou,acorn,acorn_grouped,file
0,MAC000640,2012-12-13 08:00:00,0.065,8,13,12,3,2012-12-13,0,6.26,...,1.48,snow,partly-cloudy-night,0.90,Partly Cloudy,0,ToU,ACORN-E,Affluent,block_16
1,MAC000320,2012-10-19 10:00:00,0.133,10,19,10,4,2012-10-19,0,11.78,...,1.20,rain,partly-cloudy-day,0.93,Mostly Cloudy,0,Std,ACORN-E,Affluent,block_16
2,MAC000525,2012-12-08 23:00:00,0.174,23,8,12,5,2012-12-08,1,13.84,...,4.30,rain,partly-cloudy-night,0.85,Partly Cloudy,0,ToU,ACORN-E,Affluent,block_16
3,MAC000544,2013-07-17 21:00:00,0.177,21,17,7,2,2013-07-17,0,14.39,...,1.69,rain,clear-night,0.59,Clear,0,ToU,ACORN-E,Affluent,block_16
4,MAC000673,2014-02-10 08:30:00,0.685,8,10,2,0,2014-02-10,0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,ToU,ACORN-E,Affluent,block_16


In [45]:
print("Final cleaning...")

energy_df = energy_df.dropna()
energy_df = energy_df.reset_index(drop=True)

print("Final dataset shape:", energy_df.shape)


Final cleaning...
Final dataset shape: (4145150, 25)


In [46]:
energy_df.to_csv(BASE_PATH + "final_cleaned_dataset.csv", index=False)

print("FINAL DATASET CREATED SUCCESSFULLY")


FINAL DATASET CREATED SUCCESSFULLY


In [47]:
print("final dataset shape: ",energy_df.shape)
print("final dataset head: ",energy_df.head)

final dataset shape:  (4145150, 25)
final dataset head:  <bound method NDFrame.head of              lclid                tstp  energy  hour  day  month  weekday  \
0        MAC000640 2012-12-13 08:00:00   0.065     8   13     12        3   
1        MAC000320 2012-10-19 10:00:00   0.133    10   19     10        4   
2        MAC000525 2012-12-08 23:00:00   0.174    23    8     12        5   
3        MAC000544 2013-07-17 21:00:00   0.177    21   17      7        2   
4        MAC002202 2013-08-10 12:00:00   0.076    12   10      8        5   
...            ...                 ...     ...   ...  ...    ...      ...   
4145145  MAC004675 2013-08-25 21:00:00   0.028    21   25      8        6   
4145146  MAC004166 2013-01-09 23:00:00   0.085    23    9      1        2   
4145147  MAC004542 2012-07-03 20:00:00   0.067    20    3      7        1   
4145148  MAC002694 2012-11-30 08:00:00   0.227     8   30     11        4   
4145149  MAC000259 2013-04-25 10:00:00   0.135    10   25      4  